# 類似単語のランキング表示

In [2]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    '''類似単語の検索
    :param query: クエリ（テキスト）
    :param word_to_id: 単語から単語IDへのディクショナリ
    :param id_to_word: 単語IDから単語へのディクショナリ
    :param word_matrix: 単語ベクトルをまとめた行列。各行に対応する単語のベクトルが格納されていることを想定する
    :param top: 上位何位まで表示するか
    '''

    # ① クエリを取り出す
    if query not in word_to_id:
        print('%s is not found' % query)
        return

    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # ② コサイン類似度の算出
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # ③ コサイン類似度の結果から、その値を高い順に出力
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return


1. クエリの単語ベクトルを取り出す
2. クエリの単語ベクトルとその他の全ての単語ベクトルとのコサイン類似度をそれぞれ求める
3. コサイン類似度の結果に対して、その値が高い順に表示する

In [8]:
import sys
sys.path.append("..")
from common.util import preprocess, create_co_matrix, cos_similarity
import numpy as np

In [11]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0


「you」に対して「i」が高いことは共に人称代名詞であることから納得ができそうな結果だが、

「goodbye」が一番類似度が高いという結果については感覚的なずれが生じている。
これは今回適用したテキストが極端に小さいからという理由がある。

In [12]:
most_similar('i', word_to_id, id_to_word, C, top=5)


[query] i
 goodbye: 0.9999999858578643
 you: 0.7071067691154799
 hello: 0.49999999292893216
 say: 0.0
 and: 0.0


In [13]:
most_similar('goodbye', word_to_id, id_to_word, C, top=5)


[query] goodbye
 i: 0.9999999858578643
 you: 0.7071067691154799
 hello: 0.49999999292893216
 say: 0.0
 and: 0.0


単語を共起行列によってベクトルで表したものをコサイン類似度を使って単語を評価した。

これは本書の「カウントベース」の手法の基本となる部分である。